In [ ]:
import pandas as pd
import re
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [2]:
news = pd.read_excel("./data/NewsCategorizer.xlsx")
news.head()

,Unnamed: 0,category,headline,links,short_description,keywords
0,0,WELLNESS,143 Miles in 35 Days: Lessons Learned,https://www.huffingtonpost.com/entry/running-l...,Resting is part of training. I've confirmed wh...,running-lessons
1,1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,https://www.huffingtonpost.com/entry/talking-t...,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy
2,2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,https://www.huffingtonpost.com/entry/crenezuma...,The clock is ticking for the United States to ...,crenezumab-alzheimers-disease-drug
3,3,WELLNESS,"Oh, What a Difference She Made",https://www.huffingtonpost.com/entry/meaningfu...,"If you want to be busy, keep trying to be perf...",meaningful-life
4,4,WELLNESS,Green Superfoods,https://www.huffingtonpost.com/entry/green-sup...,"First, the bad news: Soda bread, corned beef a...",green-superfoods


In [3]:
news.drop(columns=["Unnamed: 0"], inplace=True)
news.head()

,category,headline,links,short_description,keywords
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,https://www.huffingtonpost.com/entry/running-l...,Resting is part of training. I've confirmed wh...,running-lessons
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,https://www.huffingtonpost.com/entry/talking-t...,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,https://www.huffingtonpost.com/entry/crenezuma...,The clock is ticking for the United States to ...,crenezumab-alzheimers-disease-drug
3,WELLNESS,"Oh, What a Difference She Made",https://www.huffingtonpost.com/entry/meaningfu...,"If you want to be busy, keep trying to be perf...",meaningful-life
4,WELLNESS,Green Superfoods,https://www.huffingtonpost.com/entry/green-sup...,"First, the bad news: Soda bread, corned beef a...",green-superfoods


# Task 2
preprocessing

In [4]:
# making the categories lower case
news["category"] = news.category.str.lower()

In [5]:
news.short_description[0]

"Resting is part of training. I've confirmed what I sort of already knew: I'm not built for running streaks. I'm built for hard workouts three to five days a week with lots of cross training, physical therapy and foam rolling. But I've also confirmed that I'm stubborn with myself."

In [10]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()

# preprocessing
def pre_process_desc(text: str) -> str:
    text_processed = re.sub(r"\s+", " ", text) # remove all other space characters
    text_processed = re.sub(r"[^\w ]", "", text_processed).split(" ") # remove special characters and then split
    text_processed = [lemmatizer.lemmatize(word) for word in text_processed] # lemmatize the words
    text_processed = [word for word in text_processed if word not in stopwords.words("english")] # remove stopwords 
    return " ".join(text_processed)

In [11]:
# testing preprocessing
pre_process_desc(news.short_description[0])

'Resting part training Ive confirmed I sort already knew Im built running streak Im built hard workout three five day week lot cross training physical therapy foam rolling But Ive also confirmed Im stubborn'

In [12]:
# apply preprocessing
news["pre_processed_short_description"] = news.apply(lambda row: pre_process_desc(row["short_description"]), axis = 1) 

KeyboardInterrupt: 

# Task 3
fitting the lda model

In [ ]:
# common_dictionary = Dictionary(common_texts)
# common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
# >>>
# # Train the model on the corpus.
# lda = LdaModel(common_corpus, num_topics=10
news_dictionary = Dictionary(news.pre_processed_short_description.to_list())
news_corpus = [news_dictionary.doc2bow(text) for text in news.pre_processed_short_description.to_list()]

lda = LdaModel(news_corpus, num_topics = len(news.categories.unique()))